<a href="https://colab.research.google.com/github/Sid-2004/Research-Paper--WallStreetsBets-ML-Emotional-Sentiment/blob/main/Research_Paper_FinTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw yfinance transformers torch tensorflow keras-tcn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.0 MB/s eta 0:00:00


In [2]:
# ---------------------------
# 0. Setup: installs & imports
# ---------------------------
!pip install praw transformers sentence-transformers yfinance pytorch-lightning pytorch-forecasting==0.10.4 torchinfo
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # CPU fallback if GPU not available

# Standard imports
import os, re, time, math, json, datetime
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import networkx as nx
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import yfinance as yf


ERROR: Ignored the following versions that require a different python version: 0.10.2 Requires-Python >=3.8,<3.11; 0.10.3 Requires-Python >=3.8,<3.11; 1.0.0 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement pytorch-forecasting==0.10.4 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.9.0, 0.9.1, 0.9.2, 0.10.0, 0.10.1, 1.1.0, 1.1.1, 1.2.0, 1.3.0, 1.4.0)
ERROR: No matching distribution found for pytorch-forecasting==0.10.4
Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
!pip install praw yfinance transformers torch tensorflow keras-tcn
import praw
import pandas as pd

In [4]:
reddit = praw.Reddit(client_id="gpXjQ2OtfofKUYW-rcMJrw",
client_secret="TfeCSLEoqahvSnikIFDYXdAavE7qww",
user_agent="my_wsb_scraper/0.1 by your_momsbob_is_mine"
)
subreddit = reddit.subreddit("wallstreetbets")

# Example: Get top 100 posts from this week
posts = []
for submission in subreddit.top(limit=100, time_filter="week"):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score,
        "comments": submission.num_comments
    })

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [5]:
df = pd.DataFrame(posts)
df.head()

,title,text,score,comments
0,"$10,526 to $1,500,000 in 5 months thanks Google!",This account started the year with around $150...,24332,1231
1,My port last week vs this week,,13935,687
2,Tesla Offers Unprecedented $1 Trillion Pay Pac...,,13768,2305
3,Subprime burger loans go crazy,,11044,506
4,SeptemBEAR is here,,6110,178


In [6]:
df.tail()

,title,text,score,comments
95,Successful last week 0dte spy puts,By far my best % gain yet. Withdrew my last ga...,128,31
96,Red day but not for me. 6k to 30k and it’s jus...,Been eyeing loan depot forever! It’s my mortga...,124,43
97,$90k profit from AVGO ITM calls,I did suspect a modest gain but not $350 at ma...,124,21
98,"What Are Your Moves Tomorrow, September 09, 2025",This post contains content not supported on ol...,125,2609
99,Clawing back up the ladder. -18k lifetime.,Clawing back from years of losses,125,23


In [7]:
df.to_csv("wsb_posts.csv",index=False)
df.head()

,title,text,score,comments
0,"$10,526 to $1,500,000 in 5 months thanks Google!",This account started the year with around $150...,24332,1231
1,My port last week vs this week,,13935,687
2,Tesla Offers Unprecedented $1 Trillion Pay Pac...,,13768,2305
3,Subprime burger loans go crazy,,11044,506
4,SeptemBEAR is here,,6110,178


In [8]:
import re
df=pd.read_csv("wsb_posts.csv")

def clean_text(text):
  if not isinstance(text, str):
    return ""
    text = text.lower()                           # lowercase
    text = re.sub(r"http\S+|www\S+", "", text)    # remove links
    text = re.sub(r"[^a-z\s]", "", text)          # remove punctuation/numbers
    text = re.sub(r"\s+", " ", text).strip()      # remove extra spaces
    return text

df["clean_text"] = df["title"].astype(str) + " " + df["text"].astype(str)
df["clean_text"] = df["clean_text"].apply(clean_text)

df.head()

,title,text,score,comments,clean_text
0,"$10,526 to $1,500,000 in 5 months thanks Google!",This account started the year with around $150...,24332,1231,None
1,My port last week vs this week,NaN,13935,687,None
2,Tesla Offers Unprecedented $1 Trillion Pay Pac...,NaN,13768,2305,None
3,Subprime burger loans go crazy,NaN,11044,506,None
4,SeptemBEAR is here,NaN,6110,178,None


In [10]:
!pip install snscrape==0.4.3.20220106
import snscrape.modules.reddit as snreddit
import pandas as pd

posts = []
scraper = snreddit.RedditSearchScraper("subreddit:wallstreetbets after:2023-01-01")

for i, post in enumerate(scraper.get_items()):
    if i > 10000:  # limit to 10k posts for now
        break
    posts.append({
        "title": post.title,
        "text": getattr(post, "selftext", ""),
        "date": post.date,
        "url": post.url
    })

df = pd.DataFrame(posts)
df.to_csv("wsb_2023_today.csv", index=False)
print(f"Saved {len(df)} posts")
df.head()


ERROR:snscrape.base:Error retrieving https://api.pushshift.io/reddit/search/submission/?q=subreddit%3Awallstreetbets+after%3A2023-01-01&size=1000&sort=desc: non-200 status code
CRITICAL:snscrape.base:4 requests to https://api.pushshift.io/reddit/search/submission/?q=subreddit%3Awallstreetbets+after%3A2023-01-01&size=1000&sort=desc failed, giving up.


ScraperException: 4 requests to https://api.pushshift.io/reddit/search/submission/?q=subreddit%3Awallstreetbets+after%3A2023-01-01&size=1000&sort=desc failed, giving up.

**Reasoning**:
Initialize the PRAW Reddit object and retrieve a smaller number of top posts from the specified subreddit, then store the relevant data in a DataFrame.



In [11]:
reddit = praw.Reddit(client_id="gpXjQ2OtfofKUYW-rcMJrw",
client_secret="TfeCSLEoqahvSnikIFDYXdAavE7qww",
user_agent="my_wsb_scraper/0.1 by your_momsbob_is_mine"
)
subreddit = reddit.subreddit("wallstreetbets")

# Get a smaller number of top posts from this week
posts = []
for submission in subreddit.top(limit=50, time_filter="week"):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score,
        "comments": submission.num_comments
    })

df = pd.DataFrame(posts)
display(df.head())

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,title,text,score,comments
0,"$10,526 to $1,500,000 in 5 months thanks Google!",This account started the year with around $150...,24339,1231
1,My port last week vs this week,,13933,687
2,Tesla Offers Unprecedented $1 Trillion Pay Pac...,,13770,2305
3,Subprime burger loans go crazy,,11050,506
4,SeptemBEAR is here,,6110,178


**Reasoning**:
Define the clean_text function, concatenate the title and text columns, apply the cleaning function, and display the head of the dataframe to verify the results.



In [12]:
import re

def clean_text(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()                           # lowercase
  text = re.sub(r"http\S+|www\S+", "", text)    # remove links
  text = re.sub(r"[^a-z\s]", "", text)          # remove punctuation/numbers
  text = re.sub(r"\s+", " ", text).strip()      # remove extra spaces
  return text

df["clean_text"] = df["title"].astype(str) + " " + df["text"].astype(str)
df["clean_text"] = df["clean_text"].apply(clean_text)

display(df.head())

,title,text,score,comments,clean_text
0,"$10,526 to $1,500,000 in 5 months thanks Google!",This account started the year with around $150...,24339,1231,to in months thanks google this account starte...
1,My port last week vs this week,,13933,687,my port last week vs this week
2,Tesla Offers Unprecedented $1 Trillion Pay Pac...,,13770,2305,tesla offers unprecedented trillion pay packag...
3,Subprime burger loans go crazy,,11050,506,subprime burger loans go crazy
4,SeptemBEAR is here,,6110,178,septembear is here


## Analyze data

### Subtask:
Perform some basic analysis on the collected data.


In [13]:
#calculating and display the average score and number of comments and find the highest score and most comments
average_score = df['score'].mean()
average_comments = df['comments'].mean()

print(f"Average Score: {average_score:.2f}")
print(f"Average Comments: {average_comments:.2f}")

highest_score_post = df.loc[df['score'].idxmax()]
most_comments_post = df.loc[df['comments'].idxmax()]

print("\nPost with Highest Score:")
display(highest_score_post)

print("\nPost with Most Comments:")
display(most_comments_post)

Average Score: 2599.96
Average Comments: 265.22

Post with Highest Score:


,0
title,"$10,526 to $1,500,000 in 5 months thanks Google!"
text,This account started the year with around $150...
score,24339
comments,1231
clean_text,to in months thanks google this account starte...



Post with Most Comments:


,2
title,Tesla Offers Unprecedented $1 Trillion Pay Pac...
text,
score,13770
comments,2305
clean_text,tesla offers unprecedented trillion pay packag...
